In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


     |████████████████████████████████| 92kB 3.6MB/s 
     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 3.8MB 37.0MB/s 
     |████████████████████████████████| 450kB 38.8MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15354 sha256=dabde1ff485b6fb8c95918709f98e9b8418bc0f8cc016c4d692f7fca15820023
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables
ERROR: tensorflow 2.1.0rc2 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.3.3 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    U

In [0]:

#X=pd.read_table('/content/drive/My Drive/Colab Notebooks/DigitRecognitionMnist/t10k-images.idx3-ubyte',encoding='utf-8',error_bad_lines=False)

In [0]:
pip install keras-tuner
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras
import struct as st


filename = {'images' : '/content/drive/My Drive/Colab Notebooks/DigitRecognitionMnist/train-images.idx3-ubyte' ,'labels' : '/content/drive/My Drive/Colab Notebooks/DigitRecognitionMnist/train-labels.idx1-ubyte'}

train_imagesfile = open(filename['images'],'rb')
train_labelsfile=  open(filename['labels'],'rb')


train_imagesfile.seek(0)
magic = st.unpack('>4B',train_imagesfile.read(4))
train_labelsfile.seek(0)
magic1 = st.unpack('>4B',train_labelsfile.read(4))


nImg = st.unpack('>I',train_imagesfile.read(4))[0] #num of images
nR = st.unpack('>I',train_imagesfile.read(4))[0] #num of rows
nC = st.unpack('>I',train_imagesfile.read(4))[0] #num of column


cols= st.unpack('>I',train_labelsfile.read(4))[0] #num of Images


nBytesTotal1 = cols*1 #since each pixel data is 1 byte
y=  np.asarray(st.unpack('>'+'B'*nBytesTotal1,train_labelsfile.read(nBytesTotal1))).reshape((cols,1))

nBytesTotal = nImg*nR*nC*1 #since each pixel data is 1 byte
X= 255 - np.asarray(st.unpack('>'+'B'*nBytesTotal,train_imagesfile.read(nBytesTotal))).reshape((nImg,nR,nC))

#standardizing the input values
X=X/255
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=(1/12))


X_train=X_train.reshape(X_train.shape[0],1,X_train.shape[1],X_train.shape[2])
X_test=X_test.reshape(X_test.shape[0],1,X_test.shape[1],X_test.shape[2])

from tensorflow import keras
from keras.layers import Dense,Dropout
from keras.layers import ReLU,LeakyReLU,PReLU,ELU
from keras.models import Sequential 



def bulid_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(filters=hp.Int('Conv_1',max_value=128,min_value=16,step=16),
                                              kernel_size=hp.Choice('kernel1',values=[3,5]),
                                              strides=(1,1),
                                              activation='relu',
                                              kernel_intializer='he_uniform',
                                              input_shape=(1,X_train.shape[2],X_train.shape[3])),
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Conv2D(filters=hp.Int('Conv_2',max_value=256,min_value=32,step=32),
                                              kernel_size=hp.Choice('kernel2',values=[3,5]),
                                              strides=(1,1),
                                              activation='relu',
                                              kernel_intializer='he_uniform'),
                                              
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Conv2D(filters=hp.Int('Conv_3',max_value=1024,min_value=256,step=16),
                                              kernel_size=hp.Choice('kernel3',values=[3,5]),
                                              strides=(1,1),
                                              activation='relu',
                                              kernel_intializer='he_uniform'),
                                              
                          keras.layers.Conv2D(filters=hp.Int('Conv_4',max_value=1024,min_value=256,step=64),
                                              kernel_size=hp.Choice('kernel4',values=[3,5]),
                                              strides=(1,1),
                                              activation='relu',
                                              kernel_intializer='he_uniform'),
                                              
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Dense(units=hp.Int('dense_1',max_value=1024,min_value=16,step=16),
                                             activation='relu',
                                             kernel_intializer='he_uniform'),
                          keras.layers.Dense(units=hp.Int('dense_2',max_value=1024,min_value=16,step=16),
                                             activation='relu',
                                             kernel_intializer='he_uniform'),
                          keras.layers.Dense(units=10,
                                             activation='softmax',
                                             kernel_intializer='glorot_uniform')
  
  
  ])

  model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(hp.Choice('learningrate',values=[1e-1,1e-2,1e-3])),metrics=['accuracy'])
  return model



from kerastuner import RandomSearch
import kerastuner.engine.hyperparameters as hp  

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-49-a44a783bb2ce>", line 1, in <module>
    from kerastuner import RandomSearch
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/__init__.py", line 19, in <module>
    from kerastuner import applications
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/applications/__init__.py", line 17, in <module>
    from .resnet import HyperResNet
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/applications/resnet.py", line 19, in <module>
    from tensorflow.keras import layers
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/lib/pyt

ImportError: ignored

In [0]:
'''
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras
import struct as st


filename = {'images' : '/content/drive/My Drive/Colab Notebooks/DigitRecognitionMnist/train-images.idx3-ubyte' ,'labels' : '/content/drive/My Drive/Colab Notebooks/DigitRecognitionMnist/train-labels.idx1-ubyte'}

train_imagesfile = open(filename['images'],'rb')
train_labelsfile=  open(filename['labels'],'rb')


train_imagesfile.seek(0)
magic = st.unpack('>4B',train_imagesfile.read(4))
train_labelsfile.seek(0)
magic1 = st.unpack('>4B',train_labelsfile.read(4))


nImg = st.unpack('>I',train_imagesfile.read(4))[0] #num of images
nR = st.unpack('>I',train_imagesfile.read(4))[0] #num of rows
nC = st.unpack('>I',train_imagesfile.read(4))[0] #num of column


cols= st.unpack('>I',train_labelsfile.read(4))[0] #num of Images


nBytesTotal1 = cols*1 #since each pixel data is 1 byte
y=  np.asarray(st.unpack('>'+'B'*nBytesTotal1,train_labelsfile.read(nBytesTotal1))).reshape((cols,1))

nBytesTotal = nImg*nR*nC*1 #since each pixel data is 1 byte
X= 255 - np.asarray(st.unpack('>'+'B'*nBytesTotal,train_imagesfile.read(nBytesTotal))).reshape((nImg,nR,nC))

#standardizing the input values
X=X/255
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=(1/12))


arr = np.array(y_train)
data=arr.flatten()
df = pd.DataFrame()
df['label'] = (data)
y_train=df['label']


X_train=X_train.reshape(X_train.shape[0],1,X_train.shape[1],X_train.shape[2])
X_test=X_test.reshape(X_test.shape[0],1,X_test.shape[1],X_test.shape[2])

from tensorflow import keras
from keras.layers import Dense,Dropout
from keras.layers import ReLU,LeakyReLU,PReLU,ELU
from keras.models import Sequential 



def bulid_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(filters=64,
                                              kernel_size=(3,3),
                                              strides=(1,1),
                                              activation='relu',
                                              kernel_initializer='he_uniform',
                                              input_shape=(1,28,28),
                                              data_format='channels_first'),
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Conv2D(filters=128,
                                              kernel_size=(3,3),
                                              strides=(1,1),
                                              activation='relu',
                                              kernel_initializer='he_uniform',
                                              data_format='channels_first'),
                                              
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Conv2D(filters=256,
                                              kernel_size=(3,3),
                                              strides=(1,1),
                                              activation='relu',
                                              kernel_initializer='he_uniform',
                                               data_format='channels_first'),
                                              
                          keras.layers.Conv2D(filters=256,
                                              kernel_size=(3,3),
                                              strides=(1,1),
                                              activation='relu',
                                              kernel_initializer='he_uniform',
                                              data_format='channels_first'),
                                              
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Flatten(),
                          keras.layers.Dense(units=1024,
                                             activation='relu',
                                             input_dim=256,
                                             kernel_initializer='he_uniform'),
                          keras.layers.Dense(units=1024,
                                             input_dim=1024,
                                             activation='relu',
                                             kernel_initializer='he_uniform'),
                          keras.layers.Dense(units=10,
                                             input_dim=1024,
                                             activation='softmax',
                                             kernel_initializer='glorot_uniform')
  
  
  ])

  model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(hp.Choice('learningrate',values=[1e-1,1e-2,1e-3])),metrics=['accuracy'])
  model.summary()
  return model



from kerastuner import RandomSearch
import kerastuner.engine.hyperparameters as hp  

tuner_search=RandomSearch(bulid_model,
                          objective='val_accuracy',
                          max_trials=5
                          )

tuner_search.search(X_train,y_train,epochs=3,validation_split=0.1)
model=tuner_search.get_best_models(num_models=1)[0]
model.fit(X_train,y_train,epochs=10,initial_epoch=3,validation_split=0.1)


y_pred=model.predict_classes(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix
score=accuracy_score(y_pred,y_test)
cm=confusion_matrix(y_pred,y_test)


#loadig my own handwriiten imagr from the drive
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
img=load_img('/content/drive/My Drive/Colab Notebooks/DigitRecognitionMnist/IMG_20191231_111452.jpg',target_size=(28, 28))

img = np.expand_dims(img, axis=0)
img = img.reshape(img.shape[1:])
b=[0.3,0.59,0.11]
img=np.dot(img,b)
img=img.reshape(1,1,img.shape[0],img.shape[1])
img=img/255.0

y_got=model.predict_classes(img)



#creating more images for better training
img=load_img('/content/drive/My Drive/Colab Notebooks/DigitRecognitionMnist/IMG_20191231_111452.jpg',target_size=(28, 28))
x=img_to_array(img)
x=x.reshape((1,)+x.shape)
i=0
for batch in datagen.flow(x,batch_size=1,save_to_dir='/content/drive/My Drive/Colab Notebooks/DigitRecognitionMnist',save_format='jpeg'):
  i=i+1
  if(i>20):
    break
    '''




In [0]:
!pip install keras-tuner
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras

X_test=pd.read_csv('/content/drive/My Drive/Colab Notebooks/DigitRecognitionMnist/test.csv')
#X_val=pd.read_csv("/content/drive/My Drive/Colab Notebooks/miteshsir/val.csv")
X_train=pd.read_csv('/content/drive/My Drive/Colab Notebooks/DigitRecognitionMnist/train.csv')

from sklearn.model_selection import train_test_split
X_train,X_val=train_test_split(X_train)



y_train=X_train['label']
y_val=X_val['label']


#X_test=X_test.drop(['id'],axis=1)
X_train=X_train.drop(['label'],axis=1)
X_val=X_val.drop(['label'],axis=1)


X_train=X_train.to_numpy(dtype=int)
X_test=X_test.to_numpy(dtype=int)
X_val=X_val.to_numpy(dtype=int)



mtest=X_test.shape[0]
n=X_test.shape[1]
mtrain=X_train.shape[0]
mval=X_val.shape[0]
import math
n_test=math.sqrt(n)


'''from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)
X_val=sc.fit_transform(X_val)'''

X_train=X_train/255
X_test=X_test/255
X_val=X_val/255

X_train=X_train.reshape((int(mtrain),1,int(n_test),int(n_test)))
X_test=X_test.reshape((int(mtest),1,int(n_test),int(n_test)))
X_val=X_val.reshape((int(mval),1,int(n_test),int(n_test)))
#X_train=np.vstack((X_train,X_val))


from keras.models import Sequential
from keras.layers import Conv2D,Dense,Flatten,MaxPool2D,Dropout
from keras.layers import PReLU,LeakyReLU,ELU
from keras import backend as K

K.common.image_dim_ordering()
K.common.set_image_dim_ordering(dim_ordering='th')

def build_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(filters=64,
                                              #kernel_initializer='he_uniform',
                                              strides=(1,1),
                                              activation='relu',
                                              input_shape=(1,int(n_test),int(n_test)),
                                              kernel_size=(3,3),
                                              data_format='channels_first'
                                              ),
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Dropout(0.2),

                          keras.layers.Conv2D(kernel_size=(3,3),
                                              kernel_initializer='he_uniform',
                                              strides=(1,1),
                                              activation='relu',
                                              data_format='channels_first',
                                              filters=128
                                              ),
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Dropout(0.2),
                          

                          keras.layers.Conv2D(filters=256,
                                              kernel_size=(3,3),
                                              kernel_initializer='he_uniform',
                                              strides=(1,1),
                                              activation='relu',
                                              data_format='channels_first'
                                              ),
                          keras.layers.Conv2D(filters=256,
                                              kernel_size=(3,3),
                                              kernel_initializer='he_uniform',
                                              strides=(1,1),
                                              activation='relu',
                                              data_format='channels_first'
                                              ),
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Flatten(),
                          keras.layers.Dense(units=1024,input_dim=256,activation='relu',kernel_initializer='he_uniform'),
                          keras.layers.Dense(units=4096,input_dim=1024,activation='relu',kernel_initializer='he_uniform'),
                          keras.layers.Dense(units=4096,input_dim=4096,activation='relu',kernel_initializer='he_uniform'),

                          keras.layers.Dense(units=10,input_dim=1024,activation='softmax',kernel_initializer='glorot_uniform')

                                                  
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learningrate',values=[1e-2,1e-3,1e-1,1e-4])),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])
  model.summary()
  return model

from kerastuner import RandomSearch
import kerastuner.engine.hyperparameters as Hyperparametrs


tuner_search=RandomSearch(build_model,
                          objective='acc',
                          max_trials=5,
                          directory="save_dir"
                          )

tuner_search.search(X_train,y_train,epochs=3)

model=tuner_search.get_best_models(num_models=1)[0]
model.fit(X_train,y_train,epochs=20,initial_epoch=3,batch_size=15,validation_split=0.1)

y_pred1=model.predict_classes(X_val)
y_test=model.predict_classes(X_test)

from sklearn.metrics import accuracy_score
score1=accuracy_score(y_pred1,y_val)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_pred1,y_val)


test_id=[]
for i in range(mtest):
  test_id.append(i+1)

test_id=np.asarray(test_id)
test_id=test_id.reshape(28000,)

dataset = pd.DataFrame({'label': y_test})
test_id=pd.DataFrame({'Imageid':test_id})
submission = pd.concat([test_id, dataset], axis=1)
submission.to_csv('submission.csv')



INFO:tensorflow:Reloading Oracle from existing project save_dir/untitled_project/oracle.json
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 26, 26)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 13, 26)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 13, 26)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 11, 24)       36992     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 6, 24)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 6, 24)         0         
_____________________________

InvalidArgumentError: ignored

In [0]:
score1

0.99

In [0]:
X_test=pd.read_csv('/content/drive/My Drive/Colab Notebooks/DigitRecognitionMnist/test.csv')
#X_val=pd.read_csv("/content/drive/My Drive/Colab Notebooks/miteshsir/val.csv")
X_train=pd.read_csv('/content/drive/My Drive/Colab Notebooks/DigitRecognitionMnist/train.csv')

In [0]:
mtest

28000

In [0]:
test_id=[]
for i in range(mtest):
    test_id.append(i+1)

In [0]:
test_id=np.asarray(test_id)

In [0]:
test_id=test_id.reshape(28000,)